In [1]:
!pip install simpletransformers # for transformers wrapper for DL
!pip install ktrain # for ktrain wrapper for DL
!pip install nlpaug # for data augmantation
!pip install tqdm
!pip install tashaphyne # for stemming arabic words

In [45]:
import numpy as np
import pandas as pd

import json, sys, regex, os
import torch
import torchvision
import random
random.seed(2022)

from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import requests
import re 

import os
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pickle



from tqdm import tqdm


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter




import ktrain
from ktrain import text

from sklearn.utils import shuffle
from transformers import TFRobertaModel, RobertaConfig, RobertaTokenizerFast

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action

from tashaphyne.stemming import ArabicLightStemmer

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 
%config InlineBackend.figure_format = 'retina'

# Importing data from my Google Drive.

In [3]:
data = pd.read_csv('../input/arabic-dialects/Ara_dialect.csv', lineterminator='\n')
data.head()

In [4]:
data.shape

## EDA

In [5]:
plt.figure(figsize=(15, 6))

sns.countplot(data = data, y= 'dialect',
             order = data['dialect'].value_counts().index)

plt.title('Distribution number of dialect')
plt.show();

In [6]:
pd.DataFrame(data.dialect.value_counts()).T

**We sow `Egypt` have most number of tweets =D**

In [7]:
# number of dialects in data and all dialects

list_of_unique_dialect = list(set(data.dialect))
print(f'There are {len(data.dialect.value_counts())} of dialect kind')
print(f'the dialects are: \n{list_of_unique_dialect}')

**Sample of data for each dialect**
As we sow here, we need to doing some text cleaing to make machine understand the text, So we will meke:
* Removing emojis.
* Removing links.
* Removing mentions like @user.
* Removing hattage # .
* Removing puncituations.
* Removing duplicated characters.
* Removing new line "\n".
* Removing diacritization.
* Removing stop words.
* Just keep Arabic language.
<!-- * stemming for words. -->



In [8]:
print(data['tweets'][data['dialect'] == 'EG'][244697])
print('-'*50)

print(data['tweets'][data['dialect'] == 'PL'][83068])
print('-'*50)

print(data['tweets'][data['dialect'] == 'KW'][372056])
print('-'*50)

print(data['tweets'][data['dialect'] == 'LY'][15498])
print('-'*50)

print(data['tweets'][data['dialect'] == 'QA'][52000])
print('-'*50)

print(data['tweets'][data['dialect'] == 'JO'][180213])
print('-'*50)

print(data['tweets'][data['dialect'] == 'LB'][329947])
print('-'*50)

print(data['tweets'][data['dialect'] == 'SA'][218582])
print('-'*50)

print(data['tweets'][data['dialect'] == 'AE'][431900])
print('-'*50)


print(data['tweets'][data['dialect'] == 'BH'][458192])
print('-'*50)


print(data['tweets'][data['dialect'] == 'OM'][391170])
print('-'*50)


print(data['tweets'][data['dialect'] == 'SY'][126808])
print('-'*50)


print(data['tweets'][data['dialect'] == 'DZ'][244696])
print('-'*50)


print(data['tweets'][data['dialect'] == 'IQ'][1])
print('-'*50)


print(data['tweets'][data['dialect'] == 'SD'][405606])
print('-'*50)


print(data['tweets'][data['dialect'] == 'MA'][191750])
print('-'*50)


print(data['tweets'][data['dialect'] == 'YE'][228513])
print('-'*50)


print(data['tweets'][data['dialect'] == 'TN'][143049])
print('-'*50)

### Down Samplling

In [9]:
mini_rows = 9246

In [10]:
down_sample_tweets = []
down_sample_labels = []

# take just mini_row of each label
for dialect in list_of_unique_dialect:
    down_sample_tweets += data['tweets'][data['dialect'] == dialect][:mini_rows].values.tolist()
    down_sample_labels += [dialect] * mini_rows


In [11]:
len(down_sample_tweets), len(down_sample_labels)

In [12]:
down_sample_df = pd.DataFrame({'tweets': down_sample_tweets,
                              'dialect': down_sample_labels})

# shuffel data 
down_sample_df = down_sample_df.sample(frac=1).reset_index(drop=True)

down_sample_df.head()


## Text Pre-processing

In [13]:
arabic_stop_words = '''
،
ء
ءَ
آ
آب
آذار
آض
آل
آمينَ
آناء
آنفا
آه
آهاً
آهٍ
آهِ
أ
أبدا
أبريل
أبو
أبٌ
أجل
أجمع
أحد
أخبر
أخذ
أخو
أخٌ
أربع
أربعاء
أربعة
أربعمئة
أربعمائة
أرى
أسكن
أصبح
أصلا
أضحى
أطعم
أعطى
أعلم
أغسطس
أفريل
أفعل به
أفٍّ
أقبل
أكتوبر
أل
ألا
ألف
ألفى
أم
أما
أمام
أمامك
أمامكَ
أمد
أمس
أمسى
أمّا
أن
أنا
أنبأ
أنت
أنتم
أنتما
أنتن
أنتِ
أنشأ
أنه
أنًّ
أنّى
أهلا
أو
أوت
أوشك
أول
أولئك
أولاء
أولالك
أوّهْ
أى
أي
أيا
أيار
أيضا
أيلول
أين
أيّ
أيّان
أُفٍّ
ؤ
إحدى
إذ
إذا
إذاً
إذما
إذن
إزاء
إلى
إلي
إليكم
إليكما
إليكنّ
إليكَ
إلَيْكَ
إلّا
إمّا
إن
إنَّ
إى
إياك
إياكم
إياكما
إياكن
إيانا
إياه
إياها
إياهم
إياهما
إياهن
إياي
إيهٍ
ئ
ا
ا?
ا?ى
االا
االتى
ابتدأ
ابين
اتخذ
اثر
اثنا
اثنان
اثني
اثنين
اجل
احد
اخرى
اخلولق
اذا
اربعة
اربعون
اربعين
ارتدّ
استحال
اصبح
اضحى
اطار
اعادة
اعلنت
اف
اكثر
اكد
الآن
الألاء
الألى
الا
الاخيرة
الان
الاول
الاولى
التى
التي
الثاني
الثانية
الحالي
الذاتي
الذى
الذي
الذين
السابق
الف
اللاتي
اللتان
اللتيا
اللتين
اللذان
اللذين
اللواتي
الماضي
المقبل
الوقت
الى
الي
اليه
اليها
اليوم
اما
امام
امس
امسى
ان
انبرى
انقلب
انه
انها
او
اول
اي
ايار
ايام
ايضا
ب
بؤسا
بإن
بئس
باء
بات
باسم
بان
بخٍ
بد
بدلا
برس
بسبب
بسّ
بشكل
بضع
بطآن
بعد
بعدا
بعض
بغتة
بل
بلى
بن
به
بها
بهذا
بيد
بين
بَسْ
بَلْهَ
ة
ت
تاء
تارة
تاسع
تانِ
تانِك
تبدّل
تجاه
تحت
تحوّل
تخذ
ترك
تسع
تسعة
تسعمئة
تسعمائة
تسعون
تسعين
تشرين
تعسا
تعلَّم
تفعلان
تفعلون
تفعلين
تكون
تلقاء
تلك
تم
تموز
تينك
تَيْنِ
تِه
تِي
ث
ثاء
ثالث
ثامن
ثان
ثاني
ثلاث
ثلاثاء
ثلاثة
ثلاثمئة
ثلاثمائة
ثلاثون
ثلاثين
ثم
ثمان
ثمانمئة
ثمانون
ثماني
ثمانية
ثمانين
ثمنمئة
ثمَّ
ثمّ
ثمّة
ج
جانفي
جدا
جعل
جلل
جمعة
جميع
جنيه
جوان
جويلية
جير
جيم
ح
حاء
حادي
حار
حاشا
حاليا
حاي
حبذا
حبيب
حتى
حجا
حدَث
حرى
حزيران
حسب
حقا
حمدا
حمو
حمٌ
حوالى
حول
حيث
حيثما
حين
حيَّ
حَذارِ
خ
خاء
خاصة
خال
خامس
خبَّر
خلا
خلافا
خلال
خلف
خمس
خمسة
خمسمئة
خمسمائة
خمسون
خمسين
خميس
د
دال
درهم
درى
دواليك
دولار
دون
دونك
ديسمبر
دينار
ذ
ذا
ذات
ذاك
ذال
ذانك
ذانِ
ذلك
ذهب
ذو
ذيت
ذينك
ذَيْنِ
ذِه
ذِي
ر
رأى
راء
رابع
راح
رجع
رزق
رويدك
ريال
ريث
رُبَّ
ز
زاي
زعم
زود
زيارة
س
ساء
سابع
سادس
سبت
سبتمبر
سبحان
سبع
سبعة
سبعمئة
سبعمائة
سبعون
سبعين
ست
ستة
ستكون
ستمئة
ستمائة
ستون
ستين
سحقا
سرا
سرعان
سقى
سمعا
سنة
سنتيم
سنوات
سوف
سوى
سين
ش
شباط
شبه
شتانَ
شخصا
شرع
شمال
شيكل
شين
شَتَّانَ
ص
صاد
صار
صباح
صبر
صبرا
صدقا
صراحة
صفر
صهٍ
صهْ
ض
ضاد
ضحوة
ضد
ضمن
ط
طاء
طاق
طالما
طرا
طفق
طَق
ظ
ظاء
ظل
ظلّ
ظنَّ
ع
عاد
عاشر
عام
عاما
عامة
عجبا
عدا
عدة
عدد
عدم
عدَّ
عسى
عشر
عشرة
عشرون
عشرين
عل
علق
علم
على
علي
عليك
عليه
عليها
علًّ
عن
عند
عندما
عنه
عنها
عوض
عيانا
عين
عَدَسْ
غ
غادر
غالبا
غدا
غداة
غير
غين
ـ
ف
فإن
فاء
فان
فانه
فبراير
فرادى
فضلا
فقد
فقط
فكان
فلان
فلس
فهو
فو
فوق
فى
في
فيفري
فيه
فيها
ق
قاطبة
قاف
قال
قام
قبل
قد
قرش
قطّ
قلما
قوة
ك
كأن
كأنّ
كأيّ
كأيّن
كاد
كاف
كان
كانت
كانون
كثيرا
كذا
كذلك
كرب
كسا
كل
كلتا
كلم
كلَّا
كلّما
كم
كما
كن
كى
كيت
كيف
كيفما
كِخ
ل
لأن
لا
لا سيما
لات
لازال
لاسيما
لام
لايزال
لبيك
لدن
لدى
لدي
لذلك
لعل
لعلَّ
لعمر
لقاء
لكن
لكنه
لكنَّ
للامم
لم
لما
لمّا
لن
له
لها
لهذا
لهم
لو
لوكالة
لولا
لوما
ليت
ليرة
ليس
ليسب
م
مئة
مئتان
ما
ما أفعله
ما انفك
ما برح
مائة
ماانفك
مابرح
مادام
ماذا
مارس
مازال
مافتئ
ماي
مايزال
مايو
متى
مثل
مذ
مرّة
مساء
مع
معاذ
معه
معها
مقابل
مكانكم
مكانكما
مكانكنّ
مكانَك
مليار
مليم
مليون
مما
من
منذ
منه
منها
مه
مهما
ميم
ن
نا
نبَّا
نحن
نحو
نعم
نفس
نفسه
نهاية
نوفمبر
نون
نيسان
نيف
نَخْ
نَّ
ه
هؤلاء
ها
هاء
هاكَ
هبّ
هذا
هذه
هل
هللة
هلم
هلّا
هم
هما
همزة
هن
هنا
هناك
هنالك
هو
هي
هيا
هيهات
هيّا
هَؤلاء
هَاتانِ
هَاتَيْنِ
هَاتِه
هَاتِي
هَجْ
هَذا
هَذانِ
هَذَيْنِ
هَذِه
هَذِي
هَيْهات
و
و6
وأبو
وأن
وا
واحد
واضاف
واضافت
واكد
والتي
والذي
وان
واهاً
واو
واوضح
وبين
وثي
وجد
وراءَك
ورد
وعلى
وفي
وقال
وقالت
وقد
وقف
وكان
وكانت
ولا
ولايزال
ولكن
ولم
وله
وليس
ومع
ومن
وهب
وهذا
وهو
وهي
وَيْ
وُشْكَانَ
ى
ي
ياء
يفعلان
يفعلون
يكون
يلي
يمكن
يمين
ين
يناير
يوان
يورو
يوليو
يوم
يونيو
ّأيّان

'''

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations


label_classes = {"LB": "Lebanon","AE": "United Arab Emirates",
                 "PL": "PL","SY": "Syrian",
                 "TN": "Tunisian","BH": "Bahrain",
                 "OM": "Oman","SA": "Saudi Arabia",
                 "MA": "Morocco","DZ": "Algeria",
                 "JO": "Jordan","IQ": "Iraq",
                 "QA": "Qatar","LY": "Libya",
                 "KW": "Kuwait","YE": "Yemen",
                 "EG": "Egypt","SD": "Sudan",
                }


In [14]:
def remove_emojis(text): 
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_diacritization(text):
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    return text


def removing_arabic_stopwords(text):
    words = word_tokenize(text) # it works on one hadith not list
    stemmed_words = " ".join([word for word in words if word not in arabic_stop_words])    
    return stemmed_words


def stemming(text):
    ArListem = ArabicLightStemmer()
    words = word_tokenize(text) 
    stemed_text = " ".join([ArListem.light_stem(word) for word in words])
    return stemed_text


def clean_text(text):
    '''
        Make text lowercase, remove text in square brackets,remove links,remove punctuation
        and remove words containing numbers.
    '''

    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"

    text = re.sub('https?://\S+|www\.\S+', '', text)                # remove urls
    text = re.sub('<.*?>+', '', text)                               # remove tages
    text = re.sub(r'@[^\s]+', ' ', text)                            # Removing @user
    text = re.sub(r'#([^\s]+)', r'\1', text)                        # remove #word with word
    text = re.sub('[%s]' % re.escape(punctuations_list), '', text)  # remove punctuation
    text = re.sub('\n', '', text)                                   # remove new line
    text = re.sub(r'\s+', ' ', text)                                # Removing multiple spaces
    text = re.sub(sequencePattern, seqReplacePattern, text)         # Replace 3 or more consecutive letters by 2 letter.
    text = re.sub(r'\s*[A-Za-z]+\b', '' , text).rstrip()            # Removing English words and make right strip 
    return text


def preprocess_data(text):
    
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    
    # removing stop words
    text = removing_arabic_stopwords(text)

    # remove diacritization
    text = remove_diacritization(text)
    
    # Normalize the text 
    text = normalize_arabic(text)

    # Remove emojis
    text = remove_emojis(text)
    
    # stemming 
#     text = stemming(text)
    

    return text


In [15]:
some_sample_testing = [
             "السلام عليكم جميعاً ... ايه الاخبار؟",
             "@Nadjib__FCB ساهم في الفوز بالليغا والكأس😂😂😂😁😁😁",
             "لاتخليني اعرضك الهبوب https://t.co/p9v497dsLP",
             "@Maimunamamri99 عظم الله أجركم.. وربي يرحم موتاكم ويسكنهم فسيح جناته..",
             '''
             @1397Hma @almayassahamad @jaberalharmi قلتها لك يالسلوقي الف مرّه
            لاكن الانجاس مثلك ماتتوب

            عدتها كره وكره ثم كره
            وابشر بقافٍ يمسّك الدروب

            ياحماده والردى ساقك بخرّه
            وجيتنا يالسلح بالهرج الكذوب

            عرّةٍ من صلب عرّه وابن عرّه
            لاتخليني اعرضك الهبوب https://t.co/p9v497dsLP
             ''',
             '''مااحرم نفسي ميسي حريف ولعاب برضو ..
                مدريدي وافتخر 🇪🇸    
                ''',
                " #arabic فاطمه و فاطمة شو الفرق بينهم؟",
                "<arabic> العربيه",
                "الله أكبر..المصرييين جامدييين!",
                "@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. حيونه ووحشيه .. وتطلبون من الغرب يحترمكم ويؤمن بدينكم ولاينعتكم بالإرهاب .."

             ]

for tweet in some_sample_testing:
    cleaned_tweet = preprocess_data(tweet)
    print(cleaned_tweet)
    print('-'*100)


In [16]:
# testing
preprocess_data("الكلام علي ايه بقي ؟")

**Appling Tex-Preprocessing On Original Data**

In [17]:
%%time

data['clean_tweets'] = data['tweets'].apply(preprocess_data)

# reorde cols of data.
data = data.reindex(columns=['id', 'tweets', 'clean_tweets', 'dialect'])

# remove id and tweets cols
data = data.drop(['id', 'tweets'], 1)

# map each label with his country
data['dialect'] = data['dialect'].map(label_classes) 

# shuffel data 
data = data.sample(frac=1).reset_index(drop=True)

data.head()

**Appling Tex-Preprocessing On down sample Data**

In [18]:
%%time

down_sample_df['clean_tweets'] = down_sample_df['tweets'].apply(preprocess_data)

# reorde cols of data.
down_sample_df = down_sample_df.reindex(columns=['id', 'tweets', 'clean_tweets', 'dialect'])

# remove id and tweets cols
down_sample_df = down_sample_df.drop(['id', 'tweets'], 1)

# map each label with his country
down_sample_df['dialect'] = down_sample_df['dialect'].map(label_classes) 

# shuffel data 
down_sample_df = down_sample_df.sample(frac=1).reset_index(drop=True)

down_sample_df.head()

### Text Encoding.

In [19]:
LE = LabelEncoder()
down_sample_df['label_num'] = LE.fit_transform(down_sample_df['dialect'])
down_sample_df.head()

In [20]:
texts = down_sample_df['clean_tweets']

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   min_df = 2, 
                                   max_df = .95)

X = tfidf_vectorizer.fit_transform(texts) #features
y = down_sample_df['label_num'].values #target

print(X.shape)
print(y.shape)

### Machine Learning Model.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(data['clean_tweets'], data['dialect'],test_size = 0.2, random_state = 2022)

In [34]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)


#### MultinomialNB Model

In [21]:

# X_train, X_test, y_train, y_test = train_test_split(data['clean_tweets'], data['dialect'],test_size = 0.2, random_state = 2022)

# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)

# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


# clf = MultinomialNB().fit(X_train_tfidf, y_train)


In [22]:
# print(clf.predict(count_vect.transform(["الكلام ع ايه"])))

In [23]:
# check = [ 'والله ما قصرت',
#          'ايه دا فيه ايه احنا مش ناقصين',
#         'نحنا عنا لما بدنا نحكي مع مجموعة',
#          'شنو هذا يا صلاح',
#          ]

# print(clf.predict(count_vect.transform(check)))

**TEST**

In [35]:
%%time

MultinomialNB_clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = MultinomialNB_clf.predict(X_test_counts)

In [36]:
check = [ 'والله ما قصرت',
         'ايه دا فيه ايه احنا مش ناقصين',
        'نحنا عنا لما بدنا نحكي مع مجموعة',
         'شنو هذا يا صلاح',
         ]

print(MultinomialNB_clf.predict(count_vect.transform(check)))

In [37]:
classification_report_ = classification_report(y_test, y_pred)

print('MultinomialNB')
print(classification_report_)

#### LinearSVC Model

In [38]:
%%time

LinearSVC_clf = LinearSVC().fit(X_train_tfidf, y_train)
y_pred = LinearSVC_clf.predict(X_test_counts)

In [39]:
check = [ 'والله ما قصرت',
         'ايه دا فيه ايه احنا مش ناقصين',
        'نحنا عنا لما بدنا نحكي مع مجموعة',
         'شنو هذا يا صلاح',
         ]

print(LinearSVC_clf.predict(count_vect.transform(check)))

In [40]:
classification_report_ = classification_report(y_test, y_pred)

print('LinearSVC')
print(classification_report_)

#### LogisticRegression Model

In [47]:
%%time

LogisticRegression_clf = LogisticRegression(solver= 'lbfgs').fit(X_train_tfidf, y_train)
y_pred = LogisticRegression_clf.predict(X_test_counts)

In [48]:
check = [ 'والله ما قصرت',
         'ايه دا فيه ايه احنا مش ناقصين',
        'نحنا عنا لما بدنا نحكي مع مجموعة',
         'شنو هذا يا صلاح',
         ]

print(LogisticRegression_clf.predict(count_vect.transform(check)))

In [49]:
classification_report_ = classification_report(y_test, y_pred)

print('Logistic Regression Classifier')
print(classification_report_)

**Summary:**
As we sow here the `LinearSVC_clf` give me around `50%` accuracy.

#### Save the model

In [56]:
# Save the Modle to file in the current working directory
Pkl_Filename = "LinearSVC_model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(LinearSVC_clf, file)


In [59]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    loaded_LinearSVC_clf = pickle.load(file)

loaded_LinearSVC_clf

In [69]:
print(loaded_LinearSVC_clf.predict(count_vect.transform(["المكنه طلعت قماااش ي جدعان"])))

In [ ]:
# import time
# for i in range(10000):
#     print(i)
#     time.sleep(10)
